# Testando bibliotecas sgp4 + pymap3d 

**Links**

- https://www.scivision.co/pymap3d/html/index.html
- https://github.com/scivision/pymap3d

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv
from datetime import datetime, timedelta
from datetime import timezone

from tle_getter import get_tle, SATELLITES_IDS

iss_tle = get_tle(name='ISS')
print('TLE = \n{}\n{}\n'.format(*iss_tle))

TLE = 
1 25544U 98067A   18264.87214538 -.00000479 +00000-0 +00000-0 0  9990
2 25544 051.6411 256.8919 0004651 185.2142 246.8508 15.53744581133552



In [3]:
def propagate_satellite(satellite, at):
    date_at = at.year, at.month, at.day, at.hour, at.minute, at.second + (at.microsecond / 1000000.0)
    return satellite.propagate(*date_at)

In [4]:
satellite = twoline2rv(*iss_tle, wgs84)

In [5]:
at = datetime(year=2017, month=9, day=21, hour=1, minute=29, second=17, microsecond=23, tzinfo=timezone.utc)
r, v = propagate_satellite(satellite, at)

In [6]:
import pymap3d as pm

eci = r
fga_pos = (-15.989620, -48.044411, 1207)


res = pm.eci2aer(eci, *fga_pos, at)

az, el, slant_range = res
az = az[0]
el = el[0]

In [7]:
print('point in time = {}'.format(at))
print('position = {}'.format(r))
print('velocity = {}'.format(v))
print('fga(azimuth, elevation) = ({}, {})\n'.format(az, el))

point in time = 2017-09-21 01:29:17.000023+00:00
position = (-4244.46536805306, 2327.9416877050735, -4754.7619156858445)
velocity = (-0.8854028971443356, -7.117731507903258, -2.6939557912289396)
fga(azimuth, elevation) = (179.11624401667316, -89.84543681633713)



In [8]:
from tle_getter import get_az_el


lat, lng, alt = fga_pos
norad_id = 25544

print(get_az_el(lat, lng, alt, norad_id=norad_id))

(42.67, -56.7)
